It is the kernel that I have tried and compiled from the courses of [İbrahim Cebeci](https://www.udemy.com/user/ibrahim-cebeci-2/) (Language of the courses is Turkish: [Doğal Dil İşleme A-Z™: (NLP)](https://www.udemy.com/course/dogal-dil-isleme/)), which is has more than 6 courses on Udemy.

# Content


#### [1.StopWords - Stemmer - Count Vectorizer](#1.)
#### [2.Reg.Exp.- Lemmatization - Bag of Words](#2.)
#### [3.NLTK - Word2Vec(SkipGram,CBOW) - Glove](#3.)

![](https://iili.io/JGUWve.png)


<a id="1."></a> 
# 1.StopWords - Stemmer - Count Vectorizer

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
import nltk

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')

yorumlar = pd.read_csv('../input/restaurant-reviews/Restaurant_Reviews.csv')
yorumlar.head()

## Regular Expression
### SparceMatrix-StopWords-Stemmer

In [ ]:
#import re #regular expression
#Preprocessing (Önişleme) -- SparceMatrix-StopWords-Stemmer
derlem = []
for i in range(len(yorumlar)):
    yorum = re.sub('[^a-zA-Z]',' ',yorumlar['Review'][i])
    yorum = yorum.lower()
    yorum = yorum.split()
    yorum = [ps.stem(kelime) for kelime in yorum if not kelime in set(stopwords.words('english'))] #stopwords.words('turkish')
    yorum = ' '.join(yorum) #vektor için list değil string hali gerekli
    derlem.append(yorum)
    
derlem[0:5]

## CountVectorizer
### Bag of Words (BOW)

In [ ]:
#Feautre Extraction ( Öznitelik Çıkarımı) -- CountVectorizer
#Bag of Words (BOW)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2000) #kelımelerden en cok kullanılan kac tane alınsın 
X = cv.fit_transform(derlem).toarray() # bağımsız değişken
y = yorumlar.iloc[:,1].values # bağımlı değişken
print(X[0:5])
print(y[0:5])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# visualize number of digits classes
plt.figure(figsize=(15,7))
sns.countplot(y)
plt.title("Liked")

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,y_train)

y_pred = gnb.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
#print(cm)
print("score: ",gnb.score(X_test,y_test))

# %% cm visualization
import seaborn as sns
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize =(5,5))
sns.heatmap(cm,annot = True,linewidths=0.5,linecolor="red",fmt = ".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()


<a id="2."></a> 
# 2.Reg.Exp.- Lemmatization - Bag of Words

In [ ]:
import pandas as pd
import re
import nltk # natural language tool kit
nltk.download("stopwords")      # corpus diye bir kalsore indiriliyor
from nltk.corpus import stopwords  # sonra ben corpus klasorunden import ediyorum

import nltk as nlp
lemma = nlp.WordNetLemmatizer()

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

# %% import twitter data
data = pd.read_csv(r"../input/twitter-user-gender-classification/gender-classifier-DFE-791531.csv",encoding = "latin1")
data.head()

In [ ]:
data = pd.concat([data.gender,data.description],axis=1)
print(data.info())

In [ ]:
data.dropna(axis = 0,inplace = True)
data.gender = [1 if each == "female" else 0 for each in data.gender]
print(data.head())
print(data.info())

## Clening data 

In [ ]:
#%% clening data 
description_list = []
for description in data.description:
    # regular expression RE mesela "[^a-zA-Z]"
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()   # buyuk harftan kucuk harfe cevirme
    # description = description.split()
    # split yerine tokenizer kullanabiliriz
    # split kullanırsak "shouldn't " gibi kelimeler "should" ve "not" diye ikiye ayrılmaz ama word_tokenize() kullanirsak ayrilir
    description = nltk.word_tokenize(description)
    # stopwords (irrelavent words) gereksiz kelimeler
    description = [ word for word in description if not word in set(stopwords.words("english"))]
    # lemmatazation loved => love   gitmeyecegim = > git
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description] #[ ps.stem(word) for word in description]
    description = " ".join(description)  #vektor için list değil string hali gerekli
    description_list.append(description)
description_list[0:5]

## Bag of Words

In [ ]:
# %% bag of words
from sklearn.feature_extraction.text import CountVectorizer # bag of words yaratmak icin kullandigim metot
max_features = 5000 #kelımelerden en cok kullanılan kac tane alınsın

count_vectorizer = CountVectorizer(max_features=max_features,stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()  # x bağımsız değişken
print("en sik kullanilan {} kelimeden bazıları: {}".format(max_features,count_vectorizer.get_feature_names()[0:5]))

y = data.iloc[:,0].values   # male or female classes
x = sparce_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# visualize number of digits classes
plt.figure(figsize=(15,7))
sns.countplot(y)
plt.title("male or female classes")

## Train-Test Split

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 0)

# %% naive bayes
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)

#%% prediction
y_pred = nb.predict(x_test)

print("accuracy: ",nb.score(y_pred.reshape(-1,1),y_test))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

# %% cm visualization
import seaborn as sns
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize =(5,5))
sns.heatmap(cm,annot = True,linewidths=0.5,linecolor="red",fmt = ".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()


<a id="3."></a> 
# 3.NLTK - Word2Vec(SkipGram,CBOW) - Glove

## NLTK-Tokenize

In [ ]:
# NLTK-Tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
text = "Alan Mathison Turing was an English computer scientist, mathematician, logician, cryptanalyst, philosopher, and theoretical biologist. Turing was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general purpose computer. Turing is widely considered to be the father of theoretical computer science and artificial intelligence."

In [ ]:
text.split()[0:10]

In [ ]:
word_tokenize(text)[0:10] #kelıme tokenlestırme

In [ ]:
sent_tokenize(text)[0:10] #cumle tokenlestırme

## NLTK-StopWords

In [ ]:
# NLTK-StopWords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

text = 'Fazıl Say is a Turkish pianist and composer who was born in Ankara, described recently as "not merely a pianist of genius; but undoubtedly he will be one of the great artists of the twenty-first century".'

stopwords = stopwords.words('english')
print(stopwords[0:10])

In [ ]:
words = word_tokenize(text)
filtered_words = []
for word in words:
    if word not in stopwords:
        filtered_words.append(word)
        
filtered_words[0:10]

## NLTK-Stemmer

In [ ]:
# NLTK-Stemmer
from nltk.stem import PorterStemmer
ps = PorterStemmer()

words = ['drive', 'driving', 'driver', 'drives', 'drove', 'cats', 'children']
#words = ['Boyunluk', 'Boynu', 'Boylar', 'Boyun', 'Boy']

for w in words:
    print(ps.stem(w))
    
#kelimenin sonundaki ekler kesiliyor

## NLTK-Part of Speech Tagging

In [ ]:
# NLTK-Part of Speech Tagging
import nltk

text = 'Friedrich Wilhelm Nietzsche was a German philosopher, cultural critic, composer, poet, philologist, and a Latin and Greek scholar whose work has exerted a profound influence on Western philosophy and modern intellectual history. He began his career as a classical philologist before turning to philosophy. He became the youngest ever to hold the Chair of Classical Philology at the University of Basel in 1869 at the age of 24. Nietzsche resigned in 1879 due to health problems that plagued him most of his life; he completed much of his core writing in the following decade. In 1889 at age 44, he suffered a collapse and afterward, a complete loss of his mental faculties. He lived his remaining years in the care of his mother until her death in 1897 and then with his sister Elisabeth Förster-Nietzsche. Nietzsche died in 1900.'
tokenized = nltk.word_tokenize(text)
tokenized[0:10]

In [ ]:
"""
CC     coordinating conjunction
CD     cardinal digit
DT     determiner
EX     existential there (like: "there is" ... think of it like "there exists")
FW     foreign word
IN     preposition/subordinating conjunction
JJ     adjective 'big'
JJR    adjective, comparative 'bigger'
JJS    adjective, superlative 'biggest'
LS     list marker 1)
MD     modal could, will
NN     noun, singular 'desk'
NNS    noun plural 'desks'
NNP    proper noun, singular 'Harrison'
NNPS   proper noun, plural 'Americans'
PDT    predeterminer 'all the kids'
POS    possessive ending parent's
PRP    personal pronoun I, he, she
PRP$   possessive pronoun my, his, hers
RB     adverb very, silently,
RBR    adverb, comparative better
RBS    adverb, superlative best
RP     particle give up
TO     to go 'to' the store.
UH     interjection errrrrrrrm
VB     verb, base form take
VBD    verb, past tense took
VBG    verb, gerund/present participle taking
VBN    verb, past participle taken
VBP    verb, sing. present, non-3d take
VBZ    verb, 3rd person sing. present takes
WDT    wh-determiner which
WP     wh-pronoun who, what
WP$    possessive wh-pronoun whose
WRB    wh-abverb where, when
"""

In [ ]:
nltk.pos_tag(tokenized)[0:10]

## NLTK-named entitiy recognition

In [ ]:
# NLTK-named entitiy recognition

import nltk
text = "Steve Jobs was an American entrepreneur and business magnate. He was the chairman, chief executive officer (CEO), and a co-founder of Apple Inc., chairman and majority shareholder of Pixar, a member of The Walt Disney Company's board of directors following its acquisition of Pixar, and the founder, chairman, and CEO of NeXT. Jobs is widely recognized as a pioneer of the microcomputer revolution of the 1970s and 1980s, along with Apple co-founder Steve Wozniak. "
tokenized = nltk.word_tokenize(text)
print(tokenized[0:10])

In [ ]:
tagged = nltk.pos_tag(tokenized)
print(tagged[0:10])

In [ ]:
named_ent = nltk.ne_chunk(tagged)
print(named_ent[0:10])
#named_ent.draw()

In [ ]:
"""
NE Türü         	Örnek
ORGANIZATION    	Georgia-Pacific Corp., WHO
PERSON          	Eddy Bonte, President Obama
LOCATION        	Murray River, Mount Everest
DATE            	June, 2008-06-29
TIME            	two fifty a m, 1:30 p.m.
MONEY           	175 million Canadian Dollars, GBP 10.40
PERCENT         	twenty pct, 18.75 %
FACILITY        	Washington Monument, Stonehenge
GPE             	South East Asia, Midlothian
"""

## NLTK-Lemmatizing

In [ ]:
# NLTK-Lemmatizing
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

words = ['drive', 'driving', 'driver', 'drives', 'drove', 'cats', 'children']
for w in words:
    print(lem.lemmatize(w))
#kelimenin sözlükteki köküne iniliyor (morfolojik)

print(lem.lemmatize('drove', 'v'))

## Word2Vec

In [ ]:
#Word2Vec
import numpy as np
#numpy vektor ve matrısler uzerıne ıslem yapmamızı sağlar
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Word2Vec corpus üzerinde tüm kelimelerin üzerinden geç
# her kelimenin etrafındaki kelimeleri tahmin et
# iki kelime birbirne ne kadar sık bulunuyorsa vektöre yansıt
f = open('../input/hurriyet/hurriyet.txt', 'r', encoding='utf8')
text = f.read()
t_list = text.split('\n')

corpus = []

for cumle in t_list:
    corpus.append(cumle.split())
print(corpus[:10])

In [ ]:
model = Word2Vec(corpus, size=100, window=5, min_count=5, sg=1) #genelde size 5-300 uzunlugunda vektor olustururuz #gozetımsız ogrenme yontemı
#size 100 uzunlugunda vektor
#window sol ve sagda bakılacak kelıme sayısı
#sg 1 ise skip-gram kullanılacak, default olarak cbow kullanılıyor.
#mın_count kelıme en az kac kere gecıyorsa al 
model.wv['ankara']
# wv word vektorun kısaltılmısı
#kelime vektörü, word vector, word embedding, embedding hep aynı şeyi ifade ediyor

In [ ]:
print(model.wv.most_similar('almanya'))
#yazdıgınız kelıme kelıme haznesınde yoksa hata alırsınız.

In [ ]:
#model.save('word2vec.model')
#model = Word2Vec.load('word2vec.model')

In [ ]:
def closestwords_tsneplot(model, word):
    '''
     bu sınıf verılen modelı ıle kelımeyı alır
     kelımeye en yakın kelımelerın vektorlerını bır dızıye atarız
     TSNE ıle bu vektorlerı grafıge donustururuz
    '''
    word_vectors = np.empty((0,100)) # en yakın olanları lısteyi hazırladık
    word_labels = [word] #kelımeyı dızı halıne getırdık
    
    close_words = model.wv.most_similar(word) #yakın olan kelımeler bulundu
    
    word_vectors = np.append(word_vectors, np.array([model.wv[word]]), axis=0) #gelen kelımenın vektoru eklendı
    
    for w, _ in close_words: #w kelimenın kendısı dıgerı ıse yakınlık oranı
        word_labels.append(w)
        word_vectors = np.append(word_vectors, np.array([model.wv[w]]), axis=0) #yakın kelımelerın vektorlerı de eklendı
        #boylece  gelen kelıme ve yakın kelımeler word_labels, bunların vektorlerı ise word_vectors akta
        
    tsne = TSNE(random_state=0) #kelımeleı grafıge doken kutuphane
    Y = tsne.fit_transform(word_vectors)
    
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    
    plt.scatter(x_coords, y_coords)
    
    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(5, -2), textcoords='offset points')
        
    plt.show()

In [ ]:
closestwords_tsneplot(model, 'almanya')

## Glove

In [ ]:
def read_data(file_name):
    with open(file_name,'r') as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors # global vectors for word representations (Glove 2014)
'''
glove_input =  read_data('../input/glove6b100dtxt/glove.6B.100d.txt')
word2vec_output =  read_data('../input/glove-vec/glove.6B.100d.word2vec') #word2vec yazıyor ama glove kullanılacak word2vec yazmasının amacı gensım yuklemeyı kolaylastırma
glove2word2vec(glove_input, word2vec_output)


model = KeyedVectors.load_word2vec_format(word2vec_output, binary=False)
model['istanbul']
model.most_similar('ankara') #bu yontem word2vec vardı ama glove 6 mılyar kelıme ıcınde daha basarılı
model.most_similar(positive=['woman', 'king'], negative=['man'], topn=2)
#topn demek sadece n tane goster
model.most_similar(positive=['berlin', 'turkey'], negative=['ankara'], topn=1)
model.most_similar(positive=['teach', 'doctor'], negative=['treat'], topn=1)


'''
